# Análisis Exploratorio de la respuesta de AT&T a solicitud ARCO

En este _notebook_ se hace un análisis de la información proporcionada por _AT&T México_ como respuesta a una solicitud [ARCO](http://www.infoem.org.mx/src/htm/queEsArco.html) realizada por José Luis Soto Galindo. Pues conforme a la ley, la empresa telefónica está obligada a proporcionar al usuario todos los datos personales que almacene, pues él es el titular de ésta información.

La información que están obligados a proporcionar está definida según el reglamento como:
![reglamento](rawdata/descripcion_campos_obligados_entregar_telefonicas.png)

_AT&T_ entregó la información en un [archivo PDF](rawdata/documental-05-02-documento-pdf-responsable-23-agosto-2016.pdf) en formato de una tabla con los campos definidos según su  [respuesta](rawdata/respuesta_inai_att_soto_galindo.pdf) como:
![campos parte 1](rawdata/campos1.png)
![campos parte 2](rawdata/campos2.png)


Se plantea a petición del dueño de ésta información saber que inferencias se pueden hacer de su vida personal (viajes, contactos frecuentes, hábitos) con ésta información en manos de terceros, pues de acuerdo a un [reporte de solicitudes del IFT](rawdata/reportes_solicitudes_ift_2016.xlsx) hay solicitudes de éstos datos que no fueron realizadas por instituciones judiciales, lo que genera la preocupación de cual es el uso final que le dan a éstos datos.

In [115]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly
from dateutil.parser import parse
plotly.offline.init_notebook_mode(connected=True)
import cufflinks as cf

In [116]:
cf.go_offline()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [117]:
att = pd.read_csv("rawdata/tabula-Detalle de llamadas Soto Galindo (1).csv")
#att.loc[att["Ciudad"] == "90150001","Ciudad"]="CIUDAD DE MEXICO CUAUHTEMOC"
att["timestamp"] = (att["Fecha"] + " " + att["Hora"]).map(lambda x:parse(x,dayfirst=True))
att.sort_values(by="timestamp",inplace=True)

KeyError: 'Fecha'

In [118]:
att1 = pd.read_csv("rawdata/tabula-documental-05-02-documento-pdf-responsable-23-agosto-2016.csv")
len(att1.columns)

25

In [119]:
att2 = pd.read_csv("rawdata/tabula-Detalle de llamadas Soto Galindo (1).csv")
len(att2.columns)

25

In [120]:
att2.columns = att1.columns

In [121]:
att3 = pd.read_csv("rawdata/tabula-Detalle llamadas entrantes Soto galindo.csv")
att3.columns = att1.columns

In [122]:
att = pd.concat([att1,att2,att3])

In [123]:
#att = pd.read_csv("rawdata/tabula-documental-05-02-documento-pdf-responsable-23-agosto-2016.csv")
#att.loc[att["Ciudad"] == "90150001","Ciudad"]="CIUDAD DE MEXICO CUAUHTEMOC"
att["timestamp"] = (att["Fecha"] + " " + att["Hora"]).map(lambda x:parse(x,dayfirst=True))
att.sort_values(by="timestamp",inplace=True)

In [100]:
def homologa_numero(num):
    if  (type(num) == str or not np.isnan(num)) and len(str(num)) == 10:
        return "52" + str(num)
    else:
        return num

In [125]:
att["Número"] = att["Número"].map(homologa_numero)

In [126]:
att = att[att["Número"] != "525519634148"]
att = att[att["Número"] != 525519634148]

In [127]:
conteosEntranteSaliente = att[att["Evento"].isin(["LLAMADA ENTRANTE","LLAMADA SALIENTE","SMS"])][["Número","Evento"]].groupby(["Evento","Número"]).size()
conteosEntranteSaliente = conteosEntranteSaliente.swaplevel().unstack().fillna(0)
conteosEntranteSaliente["TOTAL"] = conteosEntranteSaliente["LLAMADA ENTRANTE"] + conteosEntranteSaliente["LLAMADA SALIENTE"] + conteosEntranteSaliente["SMS"]
conteosEntranteSaliente = conteosEntranteSaliente[conteosEntranteSaliente["TOTAL"] > 30].sort_values(by="TOTAL")

In [128]:
conteosEntranteSaliente.index
conteosIndex = conteosEntranteSaliente.index.tolist()
#Nombres extraídos de Facebook
conteosIndex[conteosIndex.index('525515905964')] = 'Patricia Amador'
conteosIndex[conteosIndex.index('525554377955')] = 'Mario Calixto'
conteosIndex[conteosIndex.index('523333688344')] = 'Monica Soto Galindo'
conteosIndex[conteosIndex.index('523318118121')] = 'Paola Soto Galindo'
conteosIndex[conteosIndex.index('525564167804')] = 'Luis Blisete'
conteosIndex[conteosIndex.index('525527684512')] = 'Sisi Mendoza'
conteosIndex[conteosIndex.index('525561220633')] = 'Arian Ddaszshintté'
conteosIndex[conteosIndex.index('525591832100')] = 'C. N. D. de los Pueblos Indígenas'
conteosEntranteSaliente.index = conteosIndex

## Relaciones personales
Como primera cuestión se plantea saber si se pueden inferir relaciones personales con los registros de operaciones proporcionados. Analizando el número de llamadas entrantes y salientes se obtiene la siguiente gráfica.

In [129]:
title = 'Cantidad de llamadas telefónicas, entrantes y salientes para los números mas frecuentes'
plotly.offline.iplot({"data":[go.Bar(y=conteosEntranteSaliente.index,x=conteosEntranteSaliente["LLAMADA ENTRANTE"],orientation="h",name="Llamadas Entrantes"),go.Bar(y=conteosEntranteSaliente.index,x=conteosEntranteSaliente["LLAMADA SALIENTE"],orientation="h",name="Llamadas Salientes"),go.Bar(y=conteosEntranteSaliente.index,x=conteosEntranteSaliente["SMS"],orientation="h",name="SMS")],"layout":{'barmode':'stack','title':title,'margin':{'l':220},'xaxis':{'title':'Cantidad de llamadas'},'yaxis':{'type':'category','title':'Número telefónico'}}})

De la cual se puede inferir que _Patricia Amador_, _Mario Calixto_ , _Mónica Soto Galindo_, _Paola Soto Galindo_ y _Luis Blisete_ (nombres obtenidos mediante búsqueda de números telefónicos en Facebook) son personas con un contacto frecuente con José Luis Soto Galindo. También se puede inferir que tiene contacto con organizaciones políticas, como la _Comisión Nacional de Desarrollo de los Pueblos Indígenas_.

## Actividad del usuario
¿Se pueden conocer hábitos del usuario por la actividad de su línea telefónica?

In [130]:
title = 'Cantidad de operaciones por fecha'
plotly.offline.iplot({"data":[go.Histogram(x=att["timestamp"])],"layout":{'title':title,"xaxis":{"title":"Fecha"},"yaxis":{"title":"Cantidad de operaciones"}}})

En la gráfica anterior se observa la cantidad de operaciones registradas por cada intervalo de dos semanas, se puede apreciar que la actividad de la línea corresponde principalmente al intervalo que inicia en agosto del 2014 y termina en marzo de 2016, y los puntos más bajos (enero 2015, julio 2015, enero 2016) corresponden a temporadas vacacionales, incluso las últimas dos no registran ninguna operación. Se podría suponer que en el periodo anterior a Julio de 2014 y posterior a Julio de 2016 no era su línea telefónica principal, o que la empresa esta omitiendo información.

In [131]:
horaDeLlamada = att[att["Evento"].isin(['LLAMADA ENTRANTE', 'LLAMADA SALIENTE'])]["timestamp"].apply(lambda x:x.hour)

In [132]:
    title="Conteo de llamadas por hora del día"
    tickvals = np.arange(0,24)
    ticktext =  [str(x) + "h" for x in tickvals]
    plotly.offline.iplot({"data":[go.Histogram(x=horaDeLlamada.values,orientation="v")],"layout":{"title":title,"yaxis":{"title":"Cantidad de llamadas"},"xaxis":{"title":"Hora","tickmode":"array","tickvals":tickvals,"ticktext":ticktext}}})

En la imagen anterior se visualiza la cantidad de llamadas registradas por cada hora del día, de donde se puede inferir el horario de descanso del usuario, así como su hora de mayor actividad.

In [133]:
ciudades = att[np.bitwise_and(att["Ciudad"] != "0",att["Ciudad"] != "90150001")]["Ciudad"].value_counts().sort_values()
plotly.offline.iplot({"data":[go.Bar(y=ciudades.index,x=ciudades.values,orientation="h")],"layout":{'title':'Cantidad de operaciones por ciudad','margin':{'pad':10,'l':400},'yaxis':{'type':'category','title':'Ciudad'},'xaxis':{'title':'Cantidad de operaciones en la ciudad'}}})

Y en ésta imagen se puede identificar la ciudad en la cual el usuario pasa la mayor parte de su tiempo, por lo que se puede inferir que vive y trabaja en la zona formada por la unión de la delegación Cuauhtémoc y la delegación Azcapotzalco, y que viaja regularmente a la ciudad de Guadalajara.

In [134]:
att[att["Evento"] != "DATOS"].groupby("Evento").size().sort_values().iplot(kind="barh",layout={'title':'Cantidad de operaciones por evento','margin':{'l':200,'pad':10},'yaxis':{'title':'Evento'},'xaxis':{'title':'Cantidad de operaciones'}})

Para la gráfica anterior se eliminó la contabilización de el evento _DATOS_, pues éste es mucho más frecuente y regular, y no aporta información sobre el comportamiento del usuario, del cuál se puede inferir que realiza más llamadas de las que recibe, muchas más aún que los mensajes de texto que escribe.

In [135]:
attLocs = att[np.bitwise_not(att["Ciudad"].isin(['90150001','0',np.nan]))].copy()
attLocs["ciudad_ant"] = attLocs.Ciudad.shift()
cambiosCiudad = attLocs[np.bitwise_and(np.bitwise_and(attLocs["ciudad_ant"].notnull(),attLocs["Ciudad"].notnull()),attLocs["Ciudad"] != attLocs["ciudad_ant"])].copy()
cambiosCiudad["cambio"] = cambiosCiudad["Ciudad"]+"-"+cambiosCiudad["ciudad_ant"]
conteosCambios = cambiosCiudad["cambio"].value_counts()
conteosBajos = conteosCambios[conteosCambios.values < 6]
conteosAltos = conteosCambios[conteosCambios.values >= 6]
cambiosBajos = cambiosCiudad[cambiosCiudad["cambio"].isin(conteosBajos.index)]
plotly.offline.iplot({"data":[go.Scatter(x=cambiosBajos["timestamp"],y=cambiosBajos["cambio"],mode="markers")],"layout":{'title':'Cambios de ciudad (realizados menos de 6 veces) contra fecha de su realización','margin':{'l':350},'yaxis':{'tickfont':{'size':8},'type':'category','title':'Número telefónico'}}})

En la gráfica anterior se visualizan los cambios de ciudad que tuvieron lugar en menos de 6 ocasiones (para no observar los viajes ordinarios dentro de la ciudad de México), con ésta información se puede inferir el itinerario de viaje del usuario en el periodo noviembre 2015 - diciembre 2016.

In [136]:
cambiosAltos = cambiosCiudad[cambiosCiudad["cambio"].isin(conteosAltos.index)].copy()
#plotly.offline.iplot({"data":[go.Scatter(x=cambiosBajos["timestamp"],y=cambiosBajos["cambio"],mode="markers")],"layout":{'margin':{'l':290},'yaxis':{'tickfont':{'size':8},'type':'category','title':'Número telefónico'}}})

cambiosAltos["diaCambio"] = cambiosAltos["timestamp"].map(lambda x:x.weekday_name)

cambiosSemanales = cambiosAltos[["cambio","diaCambio"]].groupby(["cambio","diaCambio"]).size().reset_index()

cambiosSemanalesFilt = cambiosSemanales[np.bitwise_and(cambiosSemanales["cambio"] != 'CIUDAD DE MEXICO AZCAPOTZALCO-CIUDAD DE MEXICO CUAUHTEMOC',cambiosSemanales["cambio"] != 'CIUDAD DE MEXICO CUAUHTEMOC-CIUDAD DE MEXICO AZCAPOTZALCO')].sort_values(by=0,ascending=False)

#plotly.offline.iplot({"data":[go.Scatter(y=cambiosSemanalesFilt["diaCambio"],x=cambiosSemanalesFilt["cambio"],mode="markers",marker={"size":8*np.sqrt(cambiosSemanalesFilt[0])})],"layout":{'margin':{'b':200,'r':200},"xaxis":{"tickfont":{"size":8}},"yaxis":{"categoryorder":"array","type":"category","categoryarray":["Sunday","Saturday","Friday","Thursday","Wednesday","Tuesday","Monday"]}}})
plotly.offline.iplot({"data":[go.Scatter(y=cambiosSemanalesFilt["diaCambio"],x=cambiosSemanalesFilt["cambio"],mode="markers",marker={"size":3*cambiosSemanalesFilt[0]})],"layout":{'title':'Cambios de ciudad (realizados mas de 6 veces) contra día de la semana de su realización','margin':{'b':200,'r':200},"xaxis":{"tickfont":{"size":8}},"yaxis":{"categoryorder":"array","type":"category","categoryarray":["Sunday","Saturday","Friday","Thursday","Wednesday","Tuesday","Monday"]}}})

Y por último, se puede observar la cantidad de cambios de ciudad por día de la semana, donde se observa una gran cantidad de viajes en fin de semana a guadalajara, que contrasta con la poca cantidad de viajes a la delegación Miguel Hidalgo desde Azcapotzalco y viceversa.

También se observa una cantidad importante de viajes Azcapotalco-Benito Juárez en viernes, Benito Juárez-Azcapotzalco en sábado y miércoles.

In [137]:
datos = att[att["Evento"] == "DATOS"].copy()

datos["fechadt"] = datos.apply(lambda x:parse(x["Fecha"],dayfirst=True),axis=1)

layout = {"title":"Consumo de datos por fecha","xaxis":{"title":"Fecha"},"yaxis":{"title":"Consumo (Kb)"}}
datos.fillna(0).groupby(["fechadt"]).sum()["KBs"].iplot(kind="bar",layout=layout)

En la gráfica anterior se trató de observar la actividad del usuario con su consumo de datos en KB, se observó actividad principalmente en el intervalo noviembre de 2014 a inicios de enero de 2015, y también en el intervalo finales de enero 2015 a finales de marzo de 2015, el resto de el año corresponde a operaciones de 0 KBs, es decir que se utilizó su teléfono para operaciones gratuitas en su plan de internet.



In [138]:
datos = att[att["Evento"] == "DATOS"].copy()

datos["fechadt"] = datos.apply(lambda x:parse(x["Fecha"],dayfirst=True),axis=1)

layout = {"title":"Consumo de datos por fecha","xaxis":{"title":"Fecha"},"yaxis":{"title":"Consumo (Kb)"}}
datos.fillna(0).groupby(["fechadt"]).size().iplot(kind="bar",layout=layout)

Debido a que las operaciones de datos con 0Kb no aportan nada en la imagen anterior, en ésta se visualizó el conteo de operaciones del evento _DATOS_ por cada día.

In [139]:
layout = {"barmode":"stack","title":"Cantidad de llamadas y mensajes por compañía telefónica","xaxis":{"title":"Compañía"},"yaxis":{"title":"Cantidad de operaciones"}}
comps=att[np.bitwise_and(att["Compañía"].notnull(), att["Compañía"] != "0")].groupby(["Compañía","Evento"]).size().unstack().fillna(0)
comps["totales"] = comps["LLAMADA ENTRANTE"]  + comps["LLAMADA SALIENTE"] + comps["SMS"]
comps.sort_values(by="totales",ascending=False,inplace=True)
comps.drop("totales",axis=1,inplace=True)
comps.iplot(kind="bar",layout=layout)#,barmode="stack")#,layout=layout)

En ésta imagen se visualiza la cantidad de llamadas y mensajes de texto agrupadas por la empresa de la línea interlocutora, de donde sabemos que además de la empresa que presta el servicio a José Soto (_AT&T_), las empresas _TELCEL_, _IUSACEL_, _OPCOM_ y _TELMEX_ poseen una gran cantidad de información de la línea analizada, pudiendo llegar a conclusiones sobre el comportamiento del usuario, sin ser su cliente.

In [141]:
att["dia"] = att["timestamp"].map(lambda x:x.weekday_name)

contDiaNum = att.groupby(["Número","dia"]).size()

layout = {'barmode':'stack','xaxis':{'type':'category','title':'Número telefónico'},"legend":{'traceorder':'normal'}}
contStack = contDiaNum[contDiaNum > 10].unstack().fillna(0)
contStack["total"] = contStack.sum(axis=1)
contStack.sort_values(by="total",ascending=False,inplace=True)
contStack.drop("total",axis=1,inplace=True)
conteosIndex = contStack.index.tolist()
#Nombres extraídos de Facebook
conteosIndex[conteosIndex.index('525515905964')] = 'Patricia Amador'
conteosIndex[conteosIndex.index('525554377955')] = 'Mario Calixto'
conteosIndex[conteosIndex.index('523333688344')] = 'Monica Soto Galindo'
conteosIndex[conteosIndex.index('523318118121')] = 'Paola Soto Galindo'
conteosIndex[conteosIndex.index('525564167804')] = 'Luis Blisete'
conteosIndex[conteosIndex.index('525527684512')] = 'Sisi Mendoza'
#conteosIndex[conteosIndex.index('5561220633')] = 'Arian Ddaszshintté'
conteosIndex[conteosIndex.index('525591832100')] = 'C. N. D. de los Pueblos Indígenas'
contStack.index = conteosIndex
contStack.iplot(kind="bar",barmode="stack",layout=layout)
data = [go.Bar(x=contStack.index,y=contStack["Monday"])]

Ésta imagen muestran los números con una mayor cantidad de llamadas y mensajes, agrupados por el día de la semana en que se efectuó la operación. Principalmente se observa poca comunicación en días sábado con _Paola Soto_ y _Mónica Soto_, y poca comunicación en fin de semana con _Patricia Amador_

In [143]:
att["hora"] = att["timestamp"].map(lambda x:x.hour)

contHoraNum = att.groupby(["Número","hora"]).size()

layout = {'barmode':'stack','xaxis':{'type':'category','title':'Número telefónico'},"legend":{'traceorder':'normal'}}
contStack = contHoraNum[contHoraNum > 2].unstack().fillna(0)
contStack["total"] = contStack.sum(axis=1)
contStack.sort_values(by="total",ascending=False,inplace=True)
contStack.drop("total",axis=1,inplace=True)
conteosIndex = contStack.index.tolist()
#Nombres extraídos de Facebook
conteosIndex[conteosIndex.index('525515905964')] = 'Patricia Amador'
conteosIndex[conteosIndex.index('525554377955')] = 'Mario Calixto'
conteosIndex[conteosIndex.index('523333688344')] = 'Monica Soto Galindo'
conteosIndex[conteosIndex.index('523318118121')] = 'Paola Soto Galindo'
conteosIndex[conteosIndex.index('525564167804')] = 'Luis Blisete'
conteosIndex[conteosIndex.index('525527684512')] = 'Sisi Mendoza'
#conteosIndex[conteosIndex.index('5561220633')] = 'Arian Ddaszshintté'
conteosIndex[conteosIndex.index('525591832100')] = 'C. N. D. de los Pueblos Indígenas'
contStack.index = conteosIndex
contStack.iplot(kind="bar",barmode="stack",layout=layout)
#data = [go.Bar(x=contStack.index,y=contStack["Monday"])]

In [144]:
contStackBubble = contStack.unstack().reset_index()

In [145]:
data = [go.Scatter(y=contStackBubble["level_1"],x=contStackBubble["hora"],mode="markers",marker={"size":3*np.sqrt(contStackBubble[0])})]
layout = {'title':'Número de llamadas y mensajes por línea telefónica y hora del día','barmode':'stack','margin':{'l':150},'yaxis':{'type':'category','title':'Número telefónico'},"xaxis":{"title":"Hora"},"legend":{'traceorder':'normal'}}
plotly.offline.iplot({"data":data,"layout":layout})

In [266]:
AMXdata = pd.DataFrame({'Telcel':[52917,0],'Telmex':[3895,0],'Telnor':[865,0],'Otros':[0,73089 - 57677]})
AMXdata.index = ['Grupo América Móvil','Resto de concesionarios']

layout = {'barmode':'stack','title':'América Móvil recibió 8 de cada 10 solicitudes hechas en 2017',
          "legend":{'traceorder':'normal'},'xaxis':{'title':'Concesionario'},'yaxis':{'title':'Cantidad de solicitudes'}}
AMXdata.iplot(kind='bar', barmode='stack', filename='cufflinks/stacked-bar-chart', layout=layout)

En la gráfica anterior se muestra que la gran mayoría de las solicitudes de información fueron dirigidas al grupo América Móvil

In [256]:
top5data = pd.DataFrame({'PGR (CENAPI, SEIDO, FEPADE, AIC)':[6353,8395,7271,7014],
                        'Fiscalía General de Justicia del Estado de México':[1890,2980,2679,2366],
                        'Procuraduría General de Justicia de Ciudad de México':[2300,2312,2808,2002],
                        'Fiscalía General del Estado de Veracruz':[2815,1313,1196,644],
                        'Procuraduría General de Justicia de Guanajuato':[1934,723,586,434]},
                        ['Primer semestre 2016','Segundo semestre 2016','Primer semestre 2017','Segundo semestre 2017',])

layout = {'yaxis':{'title':'Cantidad de solicitudes'},'barmode':'stack','title':'Autoridades que realizaron más solicitudes',"legend":{'traceorder':'normal'},'xaxis':{'tickfont':{'size':10},'type':'category','title':'Periodo'}}
top5data.iplot(kind='bar', barmode='stack', filename='cufflinks/stacked-bar-chart', layout=layout)

En ésta gráfica se muestran las autoridades que más solicitudes realizaron por semestre.

In [258]:
totalrequerimientosdata = pd.DataFrame({'Total de solicitudes':[38009,41381,42592,30497]},
                        ['Primer semestre 2016','Segundo semestre 2016','Primer semestre 2017','Segundo semestre 2017',])

layout = {'title':'Solicitudes totales a concesionarios',"legend":{'traceorder':'normal'},'xaxis':{'tickfont':{'size':10},'type':'category','title':'Periodo'}
         ,'yaxis':{'title':'Cantidad de solicitudes'}}
totalrequerimientosdata.iplot(kind='bar', filename='cufflinks/stacked-bar-chart', layout=layout)

En ésta gráfica se observa por semestre la cantidad total de solicitudes realizadas

In [259]:
solicRecibidas2017datos = pd.Series([41563,29297])
solicRecibidas2017geo = pd.Series([1029,1200])
solicRecibidas2017datosentregados = pd.Series([40282,27955])
solicRecibidas2017geoentregados = pd.Series([999,1149])
solicRecibidas2017datosrechazados = solicRecibidas2017datos - solicRecibidas2017datosentregados
solicRecibidas2017georechazados = solicRecibidas2017geo - solicRecibidas2017geoentregados
x = ["Primer semestre", "Segundo semestre"]

plotly.offline.iplot({"data":[
    go.Bar(width=0.4,y=solicRecibidas2017datosentregados+solicRecibidas2017geoentregados,x=x,orientation="v",name="Solicitudes de datos entregadas")
    ,go.Bar(width=0.4,y=solicRecibidas2017datosrechazados+solicRecibidas2017georechazados,x=x,orientation="v",name="Solicitudes de datos rechazadas")
    
],"layout":{'barmode':'stack','title':'Solicitudes entregadas y rechazadas en 2017',
           'xaxis':{'title':'Periodo'},
           'yaxis':{'title':'Cantidad de solicitudes'}}})

En ésta gráfica se observa la relación entre solicitudes aceptadas y rechazadas por los concesionarios a las autoridades

In [265]:
puebla = pd.Series([1584,1962])
x = ["Resto del año", "Mayo"]
plotly.offline.iplot({"data":[
    go.Bar(width=0.4,y=puebla,x=x,orientation="v",name="Resto del año")


],"layout":{'barmode':'stack','title':'Sólo en el mes de mayo de 2017, la fiscalía de Puebla realizó el 55% de requerimientos del año',
           'xaxis':{'title':'Periodo'},
           'yaxis':{'title':'Cantidad de solicitudes'}}})

En el mes de mayo, coincidiendo con la crisis de _huachicoleros_ en Puebla, se presentó el 55% de las solicitudes correspondientes al 2017

In [262]:
solicatt2017datosentregados = pd.Series([2821,2477])
solicatt2017datosrechazados = pd.Series([546,905])
x = ["Primer semestre", "Segundo semestre"]
plotly.offline.iplot({"data":[
    go.Bar(width=0.4,y=solicatt2017datosentregados,x=x,orientation="v",name="Solicitudes de datos entregadas")
    ,go.Bar(width=0.4,y=solicatt2017datosrechazados,x=x,orientation="v",name="Solicitudes de datos rechazadas")
    
],"layout":{'barmode':'stack','title':'AT&T aceptó 8 de cada 10 solicitudes de autoridades no identificadas',
           'xaxis':{'title':'Periodo'},
           'yaxis':{'title':'Cantidad de solicitudes'}}})

AT&T presenta una aceptación de 80% de las solicitudes realizadas por autoridades